In [2]:
import os,sys
project_path = os.path.abspath(os.path.join('..'))
sys.path.append(project_path)
os.environ.get("NEWSAPI_API_KEY")

'e3aa418fa9e249afad49e408b1d3ee40'

In [8]:
# 1. โหลด module เพื่อให้ class register เข้ากับ registry
import importlib
import yaml
from pyspark.sql import SparkSession
from registry import EXTRACTOR_REGISTRY

# ✅ Lazy import (ลด overhead)
if "extract.extractor_postgres" not in globals():
    importlib.import_module("extract.extractor_postgres")

# 2. เริ่ม Spark session (ปรับ config ให้เบาและเร็วขึ้น)
spark = (
    SparkSession.builder
    .appName("Postgres-Extract-Optimized")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")   # ✅ เปิด Arrow เพื่อเร่ง pandas<->spark
    .config("spark.sql.shuffle.partitions", "1")                   # ✅ ลดจาก 200 -> 8 partition
    .config("spark.executor.memory", "2g")                         # ✅ ใช้ memory น้อยพอสำหรับ local dev
    .config("spark.driver.memory", "1g")
    .config("spark.jars", "/path/to/postgresql-42.6.0.jar")
    .getOrCreate()
)




KeyboardInterrupt: 

In [ ]:
# 3. โหลด config จาก YAML
with open("config/source_news_articles_currentsapi.yaml", "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

extract_cfg = cfg["extractor"]
extractor_cls = EXTRACTOR_REGISTRY[extract_cfg["name"]]
extractor = extractor_cls(**extract_cfg["params"])

# ✅ 4. ใช้ predicate pushdown + partition read
# (สมมุติตารางมี column tech_id ที่เป็น index)
jdbc_url = "jdbc:postgresql://localhost:5433/postgres"
props = {
    "user": "postgres",
    "password": "!NTpatn2549",
    "driver": "org.postgresql.Driver"
}

query = "(SELECT * FROM bronze.source_news_articles_newsapi WHERE createdate >= NOW() - INTERVAL '1 day') tmp"

df = (
    spark.read.format("jdbc")
    .option("url", jdbc_url)
    .option("dbtable", query)
    .option("user", props["user"])
    .option("password", props["password"])
    .option("driver", props["driver"])
    .option("partitionColumn", "sf_id")       # ✅ ดึงแบบ parallel
    .option("lowerBound", 1)
    .option("upperBound", 500000)
    .option("numPartitions", 8)
    .load()
)

# ✅ 5. show 5 แถวแรก
df.show(5)

spark.stop()